In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset=dataset.replace({True:0,False:1})

C:\Users\gowth\AppData\Local\Temp\ipykernel_56684\1492029577.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset=dataset.replace({True:0,False:1})


In [4]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [5]:
dataset['classification_yes'].value_counts()

classification_yes
0    249
1    150
Name: count, dtype: int64

In [6]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dependent=dataset['classification_yes']

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [9]:
from sklearn.model_selection import GridSearchCV
#from sklearn.naive_bayes import GaussianNB
#from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
#from sklearn.naive_bayes import CategoricalNB
#from sklearn.naive_bayes import ComplementNB
param_grid={}
classifier=GridSearchCV(BernoulliNB(),param_grid,refit=True,verbose=0,n_jobs=-1,scoring='f1_weighted')
classifier.fit(x_train,y_train)

GridSearchCV(estimator=BernoulliNB(), n_jobs=-1, param_grid={},
             scoring='f1_weighted')

In [10]:
y_pred=classifier.predict(x_test)

In [11]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)
from sklearn.metrics import classification_report
cls_report=classification_report(y_test,y_pred,output_dict=True)
table=pd.DataFrame(cls_report).transpose()
table

[[72  3]
 [ 0 45]]


,precision,recall,f1-score,support
0,1.000000,0.960,0.979592,75.000
1,0.937500,1.000,0.967742,45.000
accuracy,0.975000,0.975,0.975000,0.975
macro avg,0.968750,0.980,0.973667,120.000
weighted avg,0.976562,0.975,0.975148,120.000


In [12]:
from sklearn.metrics import f1_score
f_score=f1_score(y_test,y_pred,average='macro')
print("The best f_score value from parameter {}:".format(classifier.best_params_),f_score)

The best f_score value from parameter {}: 0.9736668861092824


In [13]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,classifier.predict_proba(x_test)[:,1])

np.float64(1.0)